In [1]:
users = {
    
}

slips = {
    
}

import xmltodict, json
import requests


def get_odds():
    URL ="http://xml.cdn.betclic.com/odds_en.xml"


    response = requests.get(URL)
    data = response.content.decode()
    o = xmltodict.parse(data)
    data_json = json.loads(json.dumps(o)) # '{"e": {"a": ["text", "text"]}}'

    m_data = {}

    for sport in data_json["sports"]["sport"]:
        for event in sport["event"]:
                event_name = event["@name"]
                m_data[event_name] = {}
                matches = event["match"]
                if type(matches)==dict:
                    matches = [matches]
                for match in matches:
                    try:
                        match_name = match["@name"],
                        match = {
                            "start_date" : match["@start_date"],
                            "bets": {
                                bet["@name"] : {
                                    choice["@name"].replace("%","").replace("Draw","X") : choice["@odd"] for choice in bet["choice"]
                                } for bet in match["bets"]["bet"]
                            }
                        }
                        m_data[event_name][match_name] = match
                    except Exception as e:
                        pass
        break
    return m_data

odds_df = get_odds()

In [ ]:
from telegram.ext import Updater, CommandHandler, CallbackQueryHandler
from telegram import InlineKeyboardButton, InlineKeyboardMarkup
## TOKEN

BOT_NAME = "dev1010_bot"

TOKEN = "897254025:AAHsm0feiMTA_7azdg2bb1Tc3nsEOFDtOvE"

## LOGGING

import logging
logging.basicConfig(level=logging.DEBUG,
                    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

def handleMarket(bot,update):
    query = update.callback_query
    chat_id = update.effective_chat.id
    
    if query.data =="credit":
        credit(bot,update)
    elif query.data=="buy":
        buy(bot,update)
    elif query.data=="spend":
        spend(bot,update)
    elif query.data[:3]=="bet":
        match, event, quote = query.data.split("#")[1:]
        query.edit_message_text(text="Hai puntato sull'evento {} con quota {}".format(*query.data.split("#")[2:]))
        if chat_id not in slips:
            slips[chat_id] = {}
        slips[chat_id][match] = {
            "event" : event,
            "quote" : quote
        }
    elif query.data[:3]=="del":
        del slips[chat_id][query.data.split("#")[1]]
        query.edit_message_text(text="Evento eliminato.")
    else:
        query.edit_message_text(text=query.data)

def viewSlip(bot,update):
    try:
        chat_id = update.effective_chat.id

        if chat_id not in slips:
            bot.send_message(chat_id=chat_id,text="Non hai creato nessuna schedina!")
        else:
            for match, info in slips[chat_id].items():
                
                keyboard = [
                    [InlineKeyboardButton("Delete", callback_data="del#{}".format(match))]
                ]

                reply_markup = InlineKeyboardMarkup(keyboard)

                update.message.reply_text("{} {} {}".format(match,info["event"],info["quote"]), reply_markup=reply_markup)
                
    except Exception as e:
        logger.error(e)
        
        
def odds(bot,update):
    try:
        chat_id = update.effective_chat.id

        for event,matches in odds_df.items():
            bot.send_message(chat_id = chat_id, text= event)
            for match,info in matches.items():
                
                keyboard = [
                    [InlineKeyboardButton("{} ({})".format(c_n,c_v), 
                                          callback_data="bet#{}#{}#{}".format(match,c_n,c_v)) for c_n,c_v in choices.items()]
                    for bet,choices in info["bets"].items()
                ]

                reply_markup = InlineKeyboardMarkup(keyboard)

                update.message.reply_text("{} {}".format(info["start_date"],match), reply_markup=reply_markup)
            break
    except Exception as e:
        logger.error(e)
        
def market(bot,update):
    keyboard = [[InlineKeyboardButton("Buy 1 Diamond", callback_data='buy'),
             InlineKeyboardButton("Spend 1 Diamond", callback_data='spend')],

            [InlineKeyboardButton("Controlla credito", callback_data='credit')]]

    reply_markup = InlineKeyboardMarkup(keyboard)

    update.message.reply_text('Cosa vuoi fare?', reply_markup=reply_markup)

def button(update, context):
    query = update.callback_query

    query.edit_message_text(text="Selected option: {}".format(query.data))
    
def buy(bot,update):
    chat_id = update.effective_chat.id
    users[chat_id] += 1
    
def spend(bot,update):
    chat_id = update.effective_chat.id
    users[chat_id] -= 1

def start(bot,update):
    chat_id = update.effective_chat.id
    if not chat_id in users:
        bot.send_message(chat_id=chat_id,text="Benvenuto nuovo utente!")
        users[chat_id] = 0
    else:
        bot.send_message(chat_id=chat_id,text="Bentornato!")
        
def credit(bot,update):
    chat_id = update.effective_chat.id
    bot.send_message(chat_id=chat_id,text="Diamanti: {}".format(users[chat_id]) if chat_id in users else "Non sei ancora registrato! Usa /start.")
    

def main():
    updater = Updater(TOKEN)
    dp = updater.dispatcher
    dp.add_handler(CommandHandler('start',start))
    dp.add_handler(CommandHandler('credit',credit))
    dp.add_handler(CommandHandler('market',market))
    dp.add_handler(CommandHandler('odds',odds))
    dp.add_handler(CommandHandler('viewSlip',viewSlip))
    dp.add_handler(CallbackQueryHandler(handleMarket))
    updater.start_polling()
    updater.idle()

main()

2019-07-08 16:40:49,888 - telegram.ext.dispatcher - DEBUG - Setting singleton dispatcher as None
2019-07-08 16:40:49,895 - JobQueue - DEBUG - JobQueue thread started
2019-07-08 16:40:49,910 - telegram.ext.updater - DEBUG - dispatcher - started
2019-07-08 16:40:49,912 - telegram.ext.updater - DEBUG - updater - started
2019-07-08 16:40:49,919 - telegram.ext.updater - DEBUG - Updater thread started (polling)
2019-07-08 16:40:49,925 - telegram.ext.updater - DEBUG - Start network loop retry bootstrap del webhook
2019-07-08 16:40:49,928 - telegram.bot - DEBUG - Entering: delete_webhook
2019-07-08 16:40:49,933 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): api.telegram.org
2019-07-08 16:40:49,957 - telegram.ext.dispatcher - DEBUG - Dispatcher started
2019-07-08 16:40:50,285 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - DEBUG - https://api.telegram.org:443 "POST /bot897254025:AAHsm0feiMTA_7azdg2bb1Tc3nsEOFDtOvE/deleteWebhook HTTP/1.1"

2019-07-08 16:41:21,666 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - DEBUG - Starting new HTTPS connection (6): api.telegram.org
2019-07-08 16:41:26,964 - telegram.ext.updater - DEBUG - Timed out getting Updates: Timed out
2019-07-08 16:41:26,971 - telegram.bot - DEBUG - Entering: get_updates
2019-07-08 16:41:26,987 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - DEBUG - Starting new HTTPS connection (7): api.telegram.org
2019-07-08 16:41:32,678 - telegram.ext.updater - DEBUG - Timed out getting Updates: Timed out
2019-07-08 16:41:32,691 - telegram.bot - DEBUG - Entering: get_updates
2019-07-08 16:41:32,702 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - DEBUG - Starting new HTTPS connection (8): api.telegram.org
2019-07-08 16:41:37,978 - telegram.ext.updater - DEBUG - Timed out getting Updates: Timed out
2019-07-08 16:41:37,985 - telegram.bot - DEBUG - Entering: get_updates
2019-07-08 16:41:37,990 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - DEBUG - S

2019-07-08 16:41:49,667 - telegram.bot - DEBUG - Entering: get_updates
2019-07-08 16:41:49,668 - telegram.ext.dispatcher - DEBUG - Processing Update: {'update_id': 360147812, 'callback_query': {'id': '1228420542601035640', 'chat_instance': '7793455983088444007', 'message': {'message_id': 425, 'date': 1562596797, 'chat': {'id': 286013945, 'type': 'private', 'first_name': 'Christian'}, 'text': "('Manchester United - Chelsea',) X (1 +1) 4.80", 'entities': [], 'caption_entities': [], 'photo': [], 'new_chat_members': [], 'new_chat_photo': [], 'delete_chat_photo': False, 'group_chat_created': False, 'supergroup_chat_created': False, 'channel_chat_created': False, 'from': {'id': 897254025, 'first_name': 'DevBot', 'is_bot': True, 'username': 'dev1010_bot'}}, 'data': "del#('Manchester United - Chelsea',)", 'from': {'id': 286013945, 'first_name': 'Christian', 'is_bot': False, 'language_code': 'it'}}}
2019-07-08 16:41:49,673 - telegram.bot - DEBUG - Entering: edit_message_text
2019-07-08 16:41:49

2019-07-08 16:41:53,914 - telegram.bot - DEBUG - Exiting: edit_message_text
2019-07-08 16:41:58,760 - telegram.ext.updater - DEBUG - Timed out getting Updates: Timed out
2019-07-08 16:41:58,764 - telegram.bot - DEBUG - Entering: get_updates
2019-07-08 16:41:58,773 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - DEBUG - Starting new HTTPS connection (10): api.telegram.org
2019-07-08 16:42:00,017 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - DEBUG - https://api.telegram.org:443 "POST /bot897254025:AAHsm0feiMTA_7azdg2bb1Tc3nsEOFDtOvE/getUpdates HTTP/1.1" 200 5117
2019-07-08 16:42:00,022 - telegram.bot - DEBUG - Getting updates: [360147816]
2019-07-08 16:42:00,025 - telegram.bot - DEBUG - [<telegram.update.Update object at 0x7efe97872898>]
2019-07-08 16:42:00,028 - telegram.bot - DEBUG - Exiting: get_updates
2019-07-08 16:42:00,030 - telegram.bot - DEBUG - Entering: get_updates
2019-07-08 16:42:00,031 - telegram.ext.dispatcher - DEBUG - Processing Update: {'update_id': 3601

2019-07-08 16:42:24,972 - telegram.ext.updater - DEBUG - Timed out getting Updates: Timed out
2019-07-08 16:42:24,980 - telegram.bot - DEBUG - Entering: get_updates
2019-07-08 16:42:24,990 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - DEBUG - Starting new HTTPS connection (14): api.telegram.org
2019-07-08 16:42:25,617 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - DEBUG - https://api.telegram.org:443 "POST /bot897254025:AAHsm0feiMTA_7azdg2bb1Tc3nsEOFDtOvE/getUpdates HTTP/1.1" 200 322
2019-07-08 16:42:25,621 - telegram.bot - DEBUG - Getting updates: [360147819]
2019-07-08 16:42:25,625 - telegram.bot - DEBUG - [<telegram.update.Update object at 0x7efe9782d3c8>]
2019-07-08 16:42:25,630 - telegram.bot - DEBUG - Exiting: get_updates
2019-07-08 16:42:25,638 - telegram.bot - DEBUG - Entering: get_updates
2019-07-08 16:42:25,639 - telegram.ext.dispatcher - DEBUG - Processing Update: {'update_id': 360147819, 'message': {'message_id': 446, 'date': 1562596945, 'chat': {'id': 2860

...

2019-07-08 16:48:29,849 - telegram.ext.updater - DEBUG - Timed out getting Updates: Timed out
2019-07-08 16:48:29,855 - telegram.bot - DEBUG - Entering: get_updates
2019-07-08 16:48:29,864 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - DEBUG - Starting new HTTPS connection (69): api.telegram.org
2019-07-08 16:48:35,113 - telegram.ext.updater - DEBUG - Timed out getting Updates: Timed out
2019-07-08 16:48:35,121 - telegram.bot - DEBUG - Entering: get_updates
2019-07-08 16:48:35,129 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - DEBUG - Starting new HTTPS connection (70): api.telegram.org
2019-07-08 16:48:40,381 - telegram.ext.updater - DEBUG - Timed out getting Updates: Timed out
2019-07-08 16:48:40,387 - telegram.bot - DEBUG - Entering: get_updates
2019-07-08 16:48:40,395 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - DEBUG - Starting new HTTPS connection (71): api.telegram.org
2019-07-08 16:48:45,666 - telegram.ext.updater - DEBUG - Timed out getting Updates: T

2019-07-08 16:50:56,233 - telegram.ext.updater - DEBUG - Timed out getting Updates: Timed out
2019-07-08 16:50:56,239 - telegram.bot - DEBUG - Entering: get_updates
2019-07-08 16:50:56,250 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - DEBUG - Starting new HTTPS connection (96): api.telegram.org
2019-07-08 16:51:01,500 - telegram.ext.updater - DEBUG - Timed out getting Updates: Timed out
2019-07-08 16:51:01,506 - telegram.bot - DEBUG - Entering: get_updates
2019-07-08 16:51:01,514 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - DEBUG - Starting new HTTPS connection (97): api.telegram.org
2019-07-08 16:51:06,797 - telegram.ext.updater - DEBUG - Timed out getting Updates: Timed out
2019-07-08 16:51:06,806 - telegram.bot - DEBUG - Entering: get_updates
2019-07-08 16:51:06,810 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - DEBUG - Starting new HTTPS connection (98): api.telegram.org
2019-07-08 16:51:12,076 - telegram.ext.updater - DEBUG - Timed out getting Updates: T

2019-07-08 16:53:24,342 - telegram.ext.updater - DEBUG - Timed out getting Updates: Timed out
2019-07-08 16:53:24,349 - telegram.bot - DEBUG - Entering: get_updates
2019-07-08 16:53:24,354 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - DEBUG - Starting new HTTPS connection (123): api.telegram.org
2019-07-08 16:53:29,649 - telegram.ext.updater - DEBUG - Timed out getting Updates: Timed out
2019-07-08 16:53:29,654 - telegram.bot - DEBUG - Entering: get_updates
2019-07-08 16:53:29,662 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - DEBUG - Starting new HTTPS connection (124): api.telegram.org
2019-07-08 16:53:34,924 - telegram.ext.updater - DEBUG - Timed out getting Updates: Timed out
2019-07-08 16:53:34,933 - telegram.bot - DEBUG - Entering: get_updates
2019-07-08 16:53:34,941 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - DEBUG - Starting new HTTPS connection (125): api.telegram.org
2019-07-08 16:53:40,184 - telegram.ext.updater - DEBUG - Timed out getting Updates

2019-07-08 16:55:52,470 - telegram.ext.updater - DEBUG - Timed out getting Updates: Timed out
2019-07-08 16:55:52,477 - telegram.bot - DEBUG - Entering: get_updates
2019-07-08 16:55:52,486 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - DEBUG - Starting new HTTPS connection (150): api.telegram.org
2019-07-08 16:55:57,835 - telegram.ext.updater - DEBUG - Timed out getting Updates: Timed out
2019-07-08 16:55:57,841 - telegram.bot - DEBUG - Entering: get_updates
2019-07-08 16:55:57,849 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - DEBUG - Starting new HTTPS connection (151): api.telegram.org
2019-07-08 16:56:03,306 - telegram.ext.updater - DEBUG - Timed out getting Updates: Timed out
2019-07-08 16:56:03,313 - telegram.bot - DEBUG - Entering: get_updates
2019-07-08 16:56:03,319 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - DEBUG - Starting new HTTPS connection (152): api.telegram.org
2019-07-08 16:56:08,575 - telegram.ext.updater - DEBUG - Timed out getting Updates

2019-07-08 16:58:22,596 - telegram.ext.updater - DEBUG - Timed out getting Updates: Timed out
2019-07-08 16:58:22,603 - telegram.bot - DEBUG - Entering: get_updates
2019-07-08 16:58:22,610 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - DEBUG - Starting new HTTPS connection (177): api.telegram.org
2019-07-08 16:58:27,886 - telegram.ext.updater - DEBUG - Timed out getting Updates: Timed out
2019-07-08 16:58:27,891 - telegram.bot - DEBUG - Entering: get_updates
2019-07-08 16:58:27,896 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - DEBUG - Starting new HTTPS connection (178): api.telegram.org
2019-07-08 16:58:33,161 - telegram.ext.updater - DEBUG - Timed out getting Updates: Timed out
2019-07-08 16:58:33,167 - telegram.bot - DEBUG - Entering: get_updates
2019-07-08 16:58:33,175 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - DEBUG - Starting new HTTPS connection (179): api.telegram.org
2019-07-08 16:58:38,478 - telegram.ext.updater - DEBUG - Timed out getting Updates